# NHL Predictive Model

With the release of puck tracking stats via NHL EDGE, opportunities arise to build predictive models based on these data. The approach taken within is to begin with a simple shot-location based comparative predictive model. Additional models of increasing complexity will be developed as necessary.

In [29]:
# Import required libraries
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import datetime as dt
import plotly.express as px

#!pip install selenium
#!pip install webdriver_manager
#!pip install jupyter_scheduler
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 

## Baseline Model: MoneyPuck Expected Value

MoneyPuck (https://moneypuck.com/index.html) publishes daily Win probabilities for every NHL game, calculated from their historical statistics. Their model consists of two submodels (Home and Away), which calculate each team's win probability independently, before being combined via the Meta model, which factors in home-ice advantage and rest time among other factors. A full description of MoneyPuck's model can be found here: https://moneypuck.com/about.htm <br>
If we assume that MoneyPuck's model produces accurate win probabilities, a simple expected value formula can be used to identify profitable bets:

$ EV = O*P(W) - 1 $

Where 
* $EV$: Expected Value (%), Predicted Profit/Loss on a given bet
* $O$: Odds (\\$ won per \\$ wagered), A team's moneyline decimal odds 
* $P(W)$: Win Probability, MoneyPuck's stated win probability

Since decimal odds indicate how much one wins per dollar wagered, we subtract the cost to play from the expected payout ($O*P(W)$). <br>
The predictions of our shot-location (NHL EDGE) model will be compared against MoneyPuck's results, both from a betting standpoint and in terms of overall accuracy. To give further insights into each model's betting performance, we will also analyze the performance of some simple betting strategies that do not make use of predictive models.

In [5]:
'''This is the initial code used to create the moneypuck_df:
    It is remaining in this notebook for posterity, along with the code for the other major dataframes


options = webdriver.ChromeOptions()  # instantiate options 
options.headless = True  # run browser in headless mode 
# instantiate driver 
driver = webdriver.Chrome(options=options) 
 
# load website 
url = 'https://moneypuck.com/index.html?date=2023-11-02' 
driver.get(url) # get the entire website content 

moneypuck_df = pd.DataFrame()
away_win_probs, home_win_probs = [], []
away_teams, home_teams = [], []
 
# select win probability table
win_table = driver.find_elements(By.ID, 'includedContent')[0] 
games = win_table.find_elements(By.TAG_NAME, "tr")
for game in games:
    win_probs = game.find_elements(By.TAG_NAME, "h2")
    teams = game.find_elements(By.TAG_NAME, "img")
    away_win_prob = win_probs[0].text
    home_win_prob = win_probs[1].text
    away_team = teams[0].get_attribute("alt")
    home_team = teams[1].get_attribute("alt")
    away_win_prob = float(away_win_prob.replace("%",""))/100 #Remove % symbol and convert to float (0-1)
    home_win_prob = float(home_win_prob.replace("%",""))/100
    away_teams.append(away_team)
    home_teams.append(home_team)
    away_win_probs.append(away_win_prob)
    home_win_probs.append(home_win_prob)
     
moneypuck_df["date"] = str(dt.date.today()) #Initial data was collected on 10/26/23
moneypuck_df["away_team"] = away_teams
moneypuck_df["away_win_prob"] = away_win_probs
moneypuck_df["home_team"] = home_teams
moneypuck_df["home_win_prob"] = home_win_probs
moneypuck_df["date"] = str(dt.date.today())
moneypuck_df.head()'''

,date,away_team,away_win_prob,home_team,home_win_prob
0,2023-11-02,TAMPA BAY LIGHTNING,0.569,COLUMBUS BLUE JACKETS,0.431
1,2023-11-02,FLORIDA PANTHERS,0.533,DETROIT RED WINGS,0.467
2,2023-11-02,CAROLINA HURRICANES,0.517,NEW YORK RANGERS,0.483
3,2023-11-02,LOS ANGELES KINGS,0.506,OTTAWA SENATORS,0.494
4,2023-11-02,NEW YORK ISLANDERS,0.521,WASHINGTON CAPITALS,0.479


Since MoneyPuck is a dynamically loaded website, we cannot simply use a get request to extract the win probabilities from the website, as they are not present in the static html. Instead we use a headless selenium browser to load a local version of the website and then scrape the loaded content from there. The process of updating each of these dataframes has been segmented into a separate Jupyter Notebook (Update_NHL_Data) scheduled to run at 11 AM every day. This notebook will simply load the csv files generated by Update_NHL_Data and proceed from there.

In [12]:
'''Initial code included for posterity, see above


odds_df = pd.DataFrame(columns=["date"])
away_odds, home_odds = [], []
away_teams, home_teams = [], []
over_under = []
away_puck_line, away_puck_odds = [], []
home_puck_line, home_puck_odds = [], []

options = webdriver.ChromeOptions()  # instantiate options 
options.headless = True  # run browser in headless mode 
# instantiate driver 
driver = webdriver.Chrome(options=options) 

driver.get('https://www.espn.com/nhl/lines') # get the entire website content

games = driver.find_elements(By.TAG_NAME, 'tr')
i = 0
for game in games:
    game_str = game.text
    if i % 3 != 0: #Skip the header of every table
        team_data = game_str.splitlines()
        odds_line = team_data[-1] #Line containing each team's odds is the last of the table
        odds_line = odds_line.split(" ")
        if len(odds_line) == 6: #The over is always listed on the away team's row
            away_teams.append(team_data[0].upper())
            away_odds.append(convert_odds(int(odds_line[2]))) #Have to account for the goalie name being split
            over_under.append(float(odds_line[3]))
            away_puck_line.append(float(odds_line[4]))
            away_puck_odds.append(convert_odds(int(odds_line[5])))
        else:
            home_teams.append(team_data[0].upper())
            home_odds.append(convert_odds(int(odds_line[2])))
            home_puck_line.append(float(odds_line[3]))
            home_puck_odds.append(convert_odds(int(odds_line[4])))
    i += 1

odds_df["away_team"] = away_teams
odds_df["away_odds"] = away_odds
odds_df["home_team"] = home_teams
odds_df["home_odds"] = home_odds
odds_df["over_under"] = over_under
odds_df["away_puck_line"] = away_puck_line
odds_df["away_puck_odds"] = away_puck_odds
odds_df["home_puck_line"] = home_puck_line #Home puck line is inverse of away puck line, column might be redundant
odds_df["home_puck_odds"] = home_puck_odds
odds_df["date"] = str(dt.date.today())
odds_df.head()'''

,date,away_team,away_odds,home_team,home_odds,over_under,away_puck_line,away_puck_odds,home_puck_line,home_puck_odds
0,2023-11-02,LOS ANGELES KINGS,1.909091,OTTAWA SENATORS,1.909091,6.5,1.5,1.363636,-1.5,3.000000
1,2023-11-02,FLORIDA PANTHERS,1.833333,DETROIT RED WINGS,2.000000,6.5,-1.5,2.850000,1.5,1.400000
2,2023-11-02,CAROLINA HURRICANES,1.952381,NEW YORK RANGERS,1.869565,5.5,1.5,1.333333,-1.5,3.100000
3,2023-11-02,NEW YORK ISLANDERS,1.833333,WASHINGTON CAPITALS,2.000000,6.5,-1.5,2.900000,1.5,1.377358
4,2023-11-02,TAMPA BAY LIGHTNING,1.606061,COLUMBUS BLUE JACKETS,2.350000,6.5,-1.5,2.500000,1.5,1.555556


In [134]:
'''Included for posterity, see above


historical_df = pd.DataFrame(columns=["date"])
away_teams, home_teams = [], []
away_scores, home_scores = [], []

hockey_scores = requests.get("https://www.hockey-reference.com/boxscores/")
soup = BeautifulSoup(hockey_scores.text)
games = soup.find_all(attrs={"class":"teams"})
for game in games: 
    teams = game.find_all("a")[::2] #Ignore the "Final", since it's not a team
    scores = game.find_all(attrs={"class":"right"})[:3:2]
    i = 0
    for team, score in zip(teams,scores):
        if i % 2 == 0:
            away_teams.append(team.get_text().upper())
            away_scores.append(score.get_text())
        else:
            home_teams.append(team.get_text().upper())
            home_scores.append(score.get_text())
        i += 1
            
historical_df["away_team"] = away_teams
historical_df["away_score"] = away_scores
historical_df["home_team"] = home_teams
historical_df["home_score"] = home_scores
historical_df["date"] = str(dt.date.today() - dt.timedelta(days=1))
historical_df.head()'''

,date,away_team,away_score,home_team,home_score
0,2023-10-26,ANAHEIM DUCKS,4,BOSTON BRUINS,3
1,2023-10-26,SEATTLE KRAKEN,2,CAROLINA HURRICANES,3
2,2023-10-26,ST. LOUIS BLUES,3,CALGARY FLAMES,0
3,2023-10-26,TORONTO MAPLE LEAFS,4,DALLAS STARS,1
4,2023-10-26,WINNIPEG JETS,4,DETROIT RED WINGS,1


### Load Data

In [24]:
moneypuck_df = pd.read_csv("Historical_Moneypuck_Predictions.csv",index_col=0)
odds_df = pd.read_csv("Historical_Odds.csv", index_col=0)
games_df = pd.read_csv("Game_Outcomes.csv", index_col=0)

## Data Manipulation

In [25]:
temp_df = moneypuck_df.merge(odds_df, on=["date","away_team","home_team"])
#Currently not using puck line, since moneypuck doesn't predict score, just win probability
temp_df.drop(columns=["over_under","away_puck_line","away_puck_odds","home_puck_line","home_puck_odds"], inplace=True)
temp_df["away_ev"] = temp_df["away_win_prob"] * temp_df["away_odds"] - 1
temp_df["home_ev"] = temp_df["home_win_prob"] * temp_df["home_odds"] - 1
temp_df

,date,away_team,away_win_prob,home_team,home_win_prob,away_odds,home_odds,away_ev,home_ev
0,2023-11-02,TAMPA BAY LIGHTNING,0.569,COLUMBUS BLUE JACKETS,0.431,1.606061,2.350000,-0.086152,0.012850
1,2023-11-02,FLORIDA PANTHERS,0.533,DETROIT RED WINGS,0.467,1.833333,2.000000,-0.022833,-0.066000
2,2023-11-02,CAROLINA HURRICANES,0.517,NEW YORK RANGERS,0.483,1.952381,1.869565,0.009381,-0.097000
3,2023-11-02,LOS ANGELES KINGS,0.506,OTTAWA SENATORS,0.494,1.909091,1.909091,-0.034000,-0.056909
4,2023-11-02,NEW YORK ISLANDERS,0.521,WASHINGTON CAPITALS,0.479,1.833333,2.000000,-0.044833,-0.042000
...,...,...,...,...,...,...,...,...,...
57,2023-11-14,VEGAS GOLDEN KNIGHTS,0.570,WASHINGTON CAPITALS,0.430,1.645161,2.350000,-0.062258,0.010500
58,2023-11-14,ARIZONA COYOTES,0.349,DALLAS STARS,0.651,3.000000,1.416667,0.047000,-0.077750
59,2023-11-14,ANAHEIM DUCKS,0.352,NASHVILLE PREDATORS,0.648,2.550000,1.540541,-0.102400,-0.001730
60,2023-11-14,TAMPA BAY LIGHTNING,0.558,ST. LOUIS BLUES,0.442,1.769231,2.100000,-0.012769,-0.071800


In [26]:
#profit_df, implement favorite, underdog, moneypuck EV strategies
profit_df = temp_df.merge(games_df, on=["date","away_team","home_team"])
profit_df["ev_profit"] = np.where(profit_df["away_ev"] > 0, np.where(profit_df["away_score"] > profit_df["home_score"], 
                                                                    profit_df["away_odds"]-1, -1), 
                                  np.where(profit_df["home_ev"] > 0, np.where(profit_df["home_score"]>profit_df["away_score"],
                                                                    profit_df["home_odds"]-1, -1), 0))
profit_df["favorite"] = np.where(profit_df["away_odds"] < profit_df["home_odds"], np.where(profit_df["away_score"] > profit_df["home_score"],
                                                                                          profit_df["away_odds"]-1,-1),
                                np.where(profit_df["away_odds"] > profit_df["home_odds"], np.where(profit_df["away_score"] < profit_df["home_score"],
                                                                                                  profit_df["home_odds"]-1,-1), 0))
profit_df["underdog"] = np.where(profit_df["away_odds"] > profit_df["home_odds"], np.where(profit_df["away_score"] > profit_df["home_score"],
                                                                                          profit_df["away_odds"]-1,-1),
                                np.where(profit_df["away_odds"] < profit_df["home_odds"], np.where(profit_df["away_score"] < profit_df["home_score"],
                                                                                                  profit_df["home_odds"]-1,-1), 0))
profit_df

,date,away_team,away_win_prob,home_team,home_win_prob,away_odds,home_odds,away_ev,home_ev,away_score,home_score,ev_profit,favorite,underdog
0,2023-11-02,TAMPA BAY LIGHTNING,0.569,COLUMBUS BLUE JACKETS,0.431,1.606061,2.350000,-0.086152,0.012850,2,4,1.350000,-1.000000,1.350000
1,2023-11-02,FLORIDA PANTHERS,0.533,DETROIT RED WINGS,0.467,1.833333,2.000000,-0.022833,-0.066000,2,0,0.000000,0.833333,-1.000000
2,2023-11-02,CAROLINA HURRICANES,0.517,NEW YORK RANGERS,0.483,1.952381,1.869565,0.009381,-0.097000,1,2,-1.000000,0.869565,-1.000000
3,2023-11-02,LOS ANGELES KINGS,0.506,OTTAWA SENATORS,0.494,1.909091,1.909091,-0.034000,-0.056909,3,2,0.000000,0.000000,0.000000
4,2023-11-02,NEW YORK ISLANDERS,0.521,WASHINGTON CAPITALS,0.479,1.833333,2.000000,-0.044833,-0.042000,3,0,0.000000,0.833333,-1.000000
5,2023-11-02,TORONTO MAPLE LEAFS,0.458,BOSTON BRUINS,0.542,1.869565,1.952381,-0.143739,0.058190,2,3,0.952381,-1.000000,0.952381
6,2023-11-02,NEW JERSEY DEVILS,0.559,MINNESOTA WILD,0.441,1.833333,2.000000,0.024833,-0.118000,5,3,0.833333,0.833333,-1.000000
7,2023-11-02,DALLAS STARS,0.416,EDMONTON OILERS,0.584,2.300000,1.666667,-0.043200,-0.026667,4,3,0.000000,-1.000000,1.300000
8,2023-11-02,MONTREAL CANADIENS,0.468,ARIZONA COYOTES,0.532,2.100000,1.769231,-0.017200,-0.058769,2,3,0.000000,0.769231,-1.000000
9,2023-11-02,NASHVILLE PREDATORS,0.465,SEATTLE KRAKEN,0.535,2.000000,1.833333,-0.070000,-0.019167,2,4,0.000000,0.833333,-1.000000


In [28]:
ev_bets = profit_df[profit_df["ev_profit"] != 0]
favorite_bets = profit_df[profit_df["favorite"] != 0]
underdog_bets = profit_df[profit_df["underdog"] != 0]

print("Average Return on Positive Expected Value Bets: {:.4%}".format(ev_bets["ev_profit"].mean()))
print("Positive EV Sample Size (Bets Placed): {}".format(len(ev_bets["ev_profit"])))
print("Average Return on Betting Favorite Bets: {:.4%}".format(favorite_bets["favorite"].mean()))
print("Betting Favorite Sample Size (Bets Placed): {}".format(len(favorite_bets["favorite"])))
print("Average Return on Underdog Bets: {:.4%}".format(underdog_bets["underdog"].mean()))
print("Underdog Sample Size (Bets Placed): {}".format(len(underdog_bets["underdog"])))

Average Return on Positive Expected Value Bets: 2.3572%
Positive EV Sample Size (Bets Placed): 28
Average Return on Betting Favorite Bets: 12.6631%
Betting Favorite Sample Size (Bets Placed): 51
Average Return on Underdog Bets: -20.7796%
Underdog Sample Size (Bets Placed): 51


In [36]:
ev_bets = ev_bets.groupby(["date"]).mean(numeric_only=True)
favorite_bets = favorite_bets.groupby(["date"]).mean(numeric_only=True)
underdog_bets = underdog_bets.groupby(["date"]).mean(numeric_only=True)

fig = px.scatter(ev_bets, x=ev_bets.index, y="ev_profit")
fig.add_scatter(x=favorite_bets.index, y=favorite_bets["favorite"], mode="markers", name="Betting Favorite")
fig.add_scatter(x=underdog_bets.index, y=underdog_bets["underdog"], mode="markers", name="Underdog")
fig.show()